The first stage to running an encryption is to get the serial and base setup. This configures the serial, the output pins for the trigger, and the buttons and switches.

In [ ]:
import sys
!{sys.executable} -m pip install pyserial

In [1]:
print("hello world")

hello world


In [2]:
import time
import serial
from time import sleep

from pynq.overlays.base import BaseOverlay
from pynq.lib.arduino import arduino_io

base = BaseOverlay('base.bit')

# trigger for the pynq on arduino pins a0 and a1
trigger_out = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 14, 'out')
trigger_in = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 15, 'in')

key = b'aesEncryptionKey'  # can change this


In [3]:
# pull in the customlibrary
%run pyaes.ipynb

In [4]:
led0 = base.leds[0] #Corresponds to LED LD0
led1 = base.leds[1] #Corresponds to LED LD1
led2 = base.leds[2] #Corresponds to LED LD2
led3 = base.leds[3] #Corresponds to LED LD3

In [5]:
sw0 = base.switches[0] #Corresponds to SW0
sw1 = base.switches[1] #Corresponds to SW1

In [6]:
btn0 = base.buttons[0] # button 0 hopefully
btn1 = base.buttons[1]
btn2 = base.buttons[2]
btn3 = base.buttons[3]

In [15]:
class CWSerial:
    def __init__(self, device):
        self.device = device
        self.connection = None

    def configure(self):
        self.connection = serial.Serial(
            self.device, 38400, timeout=None, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE, xonxoff=False)
        time.sleep(1)

    def read(self):
        return self.connection.read()
    def readline(self):
        return self.connection.readline()

cw_com = CWSerial("/dev/ttyUSB0")
cw_com.configure()

In [16]:
print("Welcome to the pynq z2 board test")
mode = 0 # mode 0 is regular input, mode 1 is file input

Welcome to the pynq z2 board test


The setup is now complete and we can begin encryptions.

In [17]:
def encrypt(key, plaintext):
    print(f"Encrypting {plaintext} with key {key}")
    encrypter = Encrypter(AESModeOfOperationECB(key))
    ciphertext = encrypter.feed(plaintext)
    ciphertext += encrypter.feed()

    #            print( repr(ciphertext))
    for letter in ciphertext:
        print('{:0>2}'.format(str(hex(letter))[2:]), end='')
    return ciphertext

In [ ]:
# switch based LED toggling for test
data= ''
key = b'aesEncryptionKey'

while(True):  # All the code below while(True) runs forever
    if btn0.read():
        print("SW1 controls whether lights and encryption or cw encryption")
        print("In encryption mode:")
        print("\t BTN0 resets the data")
        print("\t BTN1 orders input of data\n")
        print("\t BTN2 allows key input (padded to 16 characters)")
        print("\t BTN3 does encryption")
                
        print("Sample encryption and help display:")
        key = b'aesEncryptionKey'
        plaintext = b'hello, world0000'

        aes = AESModeOfOperationECB(key)
        ciphertext = aes.encrypt(plaintext)

        print( repr(ciphertext))
        decrypted = aes.decrypt(ciphertext)
        print (f"match? {decrypted == plaintext}")

    elif not sw1.read():
        led2.toggle()
        led3.toggle()
        if btn1.read():
            led1.toggle()
            led0.toggle()
            trigger_out.write(1)
            data = input()
            x = encrypt(key, str.encode(data))
            trigger_out.write(0)
            print(x)
            
    # run the input read and encryption to the cw
    else:
        inp = cw_com.read()
        print("inside cw")
        if inp:
            print(f"received a start signal of length {len(inp)}\n {inp}")
 
            key = cw_com.readline()
            data = ""
            while inp:
                data += str(cw_com.readline())
            print(f"ready to encrypt{data}")
            trigger_out.write(1)
            x = encrypt(key, str.encode(data))
            print(f"encrypted {x}")
            #sleep(.25)
            trigger_out.write(0)
            cw_com.write(x)
            print("finished encryption")
#            key = key.encode()